In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from models import Autoencoder, EncoderStack
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tensorflow.keras.utils import to_categorical

# Load and check data

In [27]:
FILE_NAME = "GEO_data_batch_corr_final"

N_LAYERS = 3
N_NODES = [2000, 1000, 500]
DROPOUT = [0.1]
BATCH_SIZE = 3
EPOCHS = 10
TEST_RATIO = 0.15

assert len(N_NODES) == N_LAYERS or len(N_NODES) == 1
assert len(DROPOUT) == N_LAYERS or len(DROPOUT) == 1

In [28]:
dataframe = pd.read_pickle('data/pd/'+FILE_NAME)
print("Loaded {} samples with {} features.".format(dataframe.shape[0], dataframe.shape[1]))
data = dataframe.values
data = normalize(data)

classes = np.random.randint(0,1, dataframe.shape[0])
classes = to_categorical(classes)

x_train, x_test, y_train, y_test = train_test_split(data, classes, test_size=TEST_RATIO)
print("Keeping {} of samples for training: {} training, {} testing.".format(TEST_RATIO, x_train.shape[0], x_test.shape[0]))

Loaded 954 samples with 6785 features.
Keeping 0.15 of samples for training: 810 training, 144 testing.


# Train encoder layers

In [ ]:
encoder_models = []

x_train_out, x_test_out = x_train, x_test
for idx, num_hidden in enumerate(N_NODES):
    print("\n##################################################################")
    print("Training layer {} with {} hidden nodes..\n".format(idx, num_hidden))
    encoder = Autoencoder(x_train_out.shape[1], num_hidden, "output/")
    recon_mse = encoder.fit(x_train_out, x_test_out, batch_size=BATCH_SIZE, num_epochs=EPOCHS)
    x_train_out = encoder.encoder_model.predict(x_train_out)
    x_test_out = encoder.encoder_model.predict(x_test_out)
    print("\nTraining losss: ", recon_mse[0])
    print("\nTesting loss: ", recon_mse[1])
    encoder_models.append(encoder)


##################################################################
Training layer 0 with 2000 hidden nodes..

Epoch 1/10
270/270 [==============================] - 60s 222ms/step - loss: 0.0377 - val_loss: 3.6847e-04
Epoch 2/10
270/270 [==============================] - 67s 247ms/step - loss: 1.0229e-04 - val_loss: 3.6565e-05
Epoch 3/10
270/270 [==============================] - 67s 246ms/step - loss: 2.2105e-05 - val_loss: 1.4406e-05
Epoch 4/10
270/270 [==============================] - 66s 243ms/step - loss: 1.1626e-05 - val_loss: 1.1865e-05
Epoch 5/10
270/270 [==============================] - 70s 259ms/step - loss: 8.0381e-06 - val_loss: 6.6854e-06
Epoch 6/10
270/270 [==============================] - 105s 389ms/step - loss: 6.6501e-06 - val_loss: 5.8621e-06
Epoch 7/10
187/270 [===================>..........] - ETA: 31s - loss: 5.2860e-06

# Train encoder stack

In [26]:
model = EncoderStack(encoder_models, 'output/')

print("\n##################################################################")
print("Training layer {} with {} hidden nodes..\n".format(idx, num_hidden))
loss_train, loss_test = model.fit(x_train, y_train, x_test, y_test, batch_size=BATCH_SIZE, num_epochs=EPOCHS)

print("\nTraining losss: ", loss_train)
print("\nTesting loss: ", loss_test)


##################################################################
Training layer 1 with 50 hidden nodes..

Epoch 1/3
33/33 [==============================] - 1s 17ms/step - loss: 0.0820 - val_loss: 0.0104
Epoch 2/3
33/33 [==============================] - 0s 11ms/step - loss: 0.0052 - val_loss: 0.0021
Epoch 3/3
33/33 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 5.5999e-04
INFO:tensorflow:Assets written to: output/20200727-031923/encoder_stack/assets

Training losss:  0.023681095

Testing loss:  0.023681173
